<img src="http://escolar.itam.mx/centro_tesis/img/logo-ITAM.png" style="width:250px">
# Machine Learning

## Filtrado de spam - Naive Bayes
Tarea 2 : Gustavo Augusto Mondragón Sánchez : 136894

### Introducción
El clasificador bayesiano fundamenta su funcionamiento en el famoso teorema de bayes: 
 $$P(A_i|B) = P(B|A_i)P(A_i) \over P(B)$$ 

La gran importancia de este teorema viene de relacionar las probabilidades condicionales de ambos eventos a priori y a posteriori. Es decir, nos muestra como los eventos observados reflejan con una probabilidad sus eventos causales.
Por ejemplo, al conocer la probabilidad de que un correo sea spam debido a que contiene x palabras, se puede inferir la probabilidad de que un correo que contiene x palabras sea spam.


### Objetivo:

* Clasificar el correo entrante de acuerdo a si es spam o no utilizando el método Naive Bayes.
* Comparar desempeño multiplicando las probabilidades y sumando los logaritmos de las probabilidades.

In [305]:
#Librerías necesarias:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from scipy.stats import norm
from sklearn import preprocessing
from random import random

### Estructura de los [datos](https://archive.ics.uci.edu/ml/datasets/Spambase) (4601 registros):
+ 58 atributos:
    * 48 continuous real [0,100], frecuencia de palabras
    * 7 continuous real [0,100], frecuencia de caracteres
    * 2 continuous integer [1,...]
    * **1 nominal {0,1}, 1 si es spam, 0 si no lo es.**
    

In [306]:
#Cargar los datos desde repositorio online:
sdata = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data",header=None) # Leer el archivo
#Cambiar el nombre a las columnas:
sdata.columns = ['freq_make', 'freq_address', 'freq_all', 'freq_3d', 'freq_our', 'freq_over', 'freq_remove', 'freq_internet', 'freq_order', 'freq_mail', 'freq_receive', 'freq_will', 'freq_people', 'freq_report', 'freq_addresses', 'freq_free', 'freq_business', 'freq_email', 'freq_you', 'freq_credit', 'freq_your', 'freq_font', 'freq_000', 'freq_money', 'freq_hp', 'freq_hpl', 'freq_george', 'freq_650', 'freq_lab', 'freq_labs', 'freq_telnet', 'freq_857', 'freq_data', 'freq_415', 'freq_85', 'freq_technology', 'freq_1999', 'freq_parts', 'freq_pm', 'freq_direct', 'freq_cs', 'freq_meeting', 'freq_original', 'freq_project', 'freq_re', 'freq_edu', 'freq_table', 'freq_conference', 'freq_;', 'freq_(', 'freq_[', 'freq_!', 'freq_$', 'freq_#', 'crl_average', 'crl_longest', 'crl_total', 'isSpam']
#Descripción previa de los datos:
sdata.describe()

,freq_make,freq_address,freq_all,freq_3d,freq_our,freq_over,freq_remove,freq_internet,freq_order,freq_mail,...,freq_;,freq_(,freq_[,freq_!,freq_$,freq_#,crl_average,crl_longest,crl_total,isSpam
count,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,...,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000
mean,0.104553,0.213015,0.280656,0.065425,0.312223,0.095901,0.114208,0.105295,0.090067,0.239413,...,0.038575,0.139030,0.016976,0.269071,0.075811,0.044238,5.191515,52.172789,283.289285,0.394045
std,0.305358,1.290575,0.504143,1.395151,0.672513,0.273824,0.391441,0.401071,0.278616,0.644755,...,0.243471,0.270355,0.109394,0.815672,0.245882,0.429342,31.729449,194.891310,606.347851,0.488698
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.588000,6.000000,35.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.065000,0.000000,0.000000,0.000000,0.000000,2.276000,15.000000,95.000000,0.000000
75%,0.000000,0.000000,0.420000,0.000000,0.380000,0.000000,0.000000,0.000000,0.000000,0.160000,...,0.000000,0.188000,0.000000,0.315000,0.052000,0.000000,3.706000,43.000000,266.000000,1.000000
max,4.540000,14.280000,5.100000,42.810000,10.000000,5.880000,7.270000,11.110000,5.260000,18.180000,...,4.385000,9.752000,4.081000,32.478000,6.003000,19.829000,1102.500000,9989.000000,15841.000000,1.000000


### División del conjunto de datos en entrenamiento y validación

In [494]:
np.random.seed(123)     #Para revisiones de los resultados futuras

i = np.random.rand(len(sdata)) < 0.75
#División de los datos 
train = sdata[i]
test = sdata[~i]

trainx = np.array(train.iloc[:,0:-1])
trainy = np.array(train.iloc[:,-1:])
testx = np.array(test.iloc[:,0:-1])
testy = np.array(test.iloc[:,-1:])

#Validar tamaño correcto:
print "Tamaño entrenamiento %f: %f%%"%(len(train),(100*float(len(train))/len(sdata)))
print "Tamaño validación %f: %f%%"%(len(test),(100*float(len(test))/len(sdata)))

Tamaño entrenamiento 3463.000000: 75.266246%
Tamaño validación 1138.000000: 24.733754%


### Calcular probabilidad del evento y medidas de tendencia central

In [355]:
#Probabilidad de spam:
pSpam = float(sum(train["isSpam"]))/len(train)
pNotSpam = 1-pSpam
print "Probabilidad de que sea Spam:%f%%"%(100*pSpam)
print "Probabilidad de que NO sea Spam:%f%%"%(100*pNotSpam)

#Obtenemos la media y la varianza de cada atributo:
#SPAM
mediaS = np.array(np.mean(train[train["isSpam"]==1]))
dsS = np.array(np.std(train[train["isSpam"]==1],0))
# NO SPAM
mediaNS = np.array(np.mean(train[train["isSpam"]==0],0))
dsNS = np.array(np.std(train[train["isSpam"]==0],0))

Probabilidad de que sea Spam:39.214554%
Probabilidad de que NO sea Spam:60.785446%


### Función de clasificaador bayesiano

In [491]:
# Calculo de los logaritmos de las probabilidades condicionales usando Naive Bayes 
def Bayes_Filter_M(x,mean_s,mean_ns,sd_s,sd_ns,p_spam): 
    p_noS = 1 - p_spam
    pred = np.zeros(len(x))
    spam = np.log(p_spam)
    nospam = np.log(p_noS)
    
    for i in range(len(x)):
        for j in range(len(mean_s)-1):
            p = norm.pdf(x[i][j],mean_s[j], sd_s[j])
            if (p != 0):
                spam += np.log(p)
            else:
                spam += -10000.0
        for k in range(len(mean_ns)-1):
            p = norm.pdf(x[i][k],mean_ns[k], sd_ns[k])
            if (p != 0):
                nospam += np.log(p)
            else:
                nospam += -10000.0
        if spam > nospam:
            pred[i]= 1
        else:
            pred[i]= 0
    return pred

### Evaluación del modelo y comparativo con sklearn

In [501]:
# Prueba el filtro 1
prediction = Bayes_Filter_M(testx,mediaS,mediaNS,dsS,dsNS,pSpam)
print "Algoritmo multiplicando probabilidades"
print confusion_matrix(testy,prediction)

c = 0
for i in range(len(testx)):
    if(prediction[i] == testy[i]):
        c = c + 1
print "Tasa de predicción: %f%%"%(100*float(c)/len(testx))

print "\n\n Algoritmo sklearn"
# Resultados con sklearn
from sklearn.naive_bayes import GaussianNB
NB=GaussianNB()
NB.fit(trainx,trainy) 
pred=NB.predict(testx) 
print(confusion_matrix(testy,pred))
c = 0
for i in range(len(testx)):
    if(pred[i] == testy[i]):
        c = c + 1
print "Tasa de predicción: %f%%"%(100*float(c)/len(testx))


Algoritmo multiplicando probabilidades
[[603  80]
 [  0 455]]
Tasa de predicción: 92.970123%


 Algoritmo sklearn
[[497 186]
 [ 22 433]]
Tasa de predicción: 81.722320%
